In [1]:
import requests


In [8]:
url = 'https://bank-price-api.herokuapp.com/getpdfs'
dic ={"1":{"url":"https://www.abanca.pt", 
                    "name":"ABANCA Corporación Bancaria, S.A.", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0170"}
             ,
              "2":{"url":"https://www.bankinter.pt", 
                    "name":"Bankinter, S.A.", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0269"}
              ,
              "3":{"url":"https://www.bancobic.ao", 
                    "name":"Banco BIC, SA", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0079"}
              }
r = requests.post(url=url , json=dic)

In [9]:
r.json()

{'1': {'bp_bank_id': '0170',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf',
  'last_updated': '210401',
  'list_pdfs': ['https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf',
   'https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf',
   'https://www.abanca.pt/files/documents/precariodevaloresmobiliarios-23c714a2.pdf',
   'https://www.abanca.pt/files/documents/glossario-6479e89f.pdf',
   'https://www.abanca.pt/files/docs/leaflet-your-rights-payments-eu.pdf'],
  'name': 'ABANCA Corporación Bancaria, S.A.',
  'num_pdfs': 5,
  'price_page': 'https://www.abanca.pt/pt/precario/',
  'sum_sizes': '40',
  'url': 'https://www.abanca.pt'},
 '2': {'bp_bank_id': '0269',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf',
  'last_updated': '210401',
  'list_pdfs': ['https://www.bankinter.pt/documents/20182/115991/Pre%C3%A7%C3%A1rio+Completo+T

# RQ background jobs

In [1]:
import requests

def count_words_at_url(url):
    resp = requests.get(url)
    return len(resp.text.split())

In [2]:
from redis import Redis
from rq import Queue

q = Queue(connection=Redis())

In [3]:
result = q.enqueue(count_words_at_url, 'http://nvie.com')

ValueError: Functions from the __main__ module cannot be processed by workers